# Create initial model

In [ ]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'quality-prediction-automl'
project_folder = './projects/quality-prediction-automl'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

# Create the train / test datasets using all the features

In [ ]:
ws = Workspace.from_config()
allData = ws.datasets['qualitydataset'].to_pandas_dataframe()

allfeatures = ['Quality','S1','S2','S3','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23','S24','S25','S26','S27','S28','S29','S30','S31','S32','S33','S34','S35','S36','S37','S38','S39','S40','S41']

trainData = allData[allfeatures]
print(trainData.shape)
trainData = trainData.dropna()
print(trainData.shape)
trainData.head(50)

X = trainData[allfeatures[1:len(allfeatures)]].astype(float).values # exclude first feature
y = trainData[allfeatures[0]].astype(float).values  # use first feature
print(X)
print(y)

automl_config = AutoMLConfig(task = 'classification',
                             name = experiment_name,
                             primary_metric = 'AUC_weighted',
                             X = X, 
                             y = y,
                             preprocess = False,
                             iterations = 10,
                             path = project_folder)

# Train the model

In [ ]:
local_run = experiment.submit(automl_config, show_output = True)

In [ ]:
local_run

# Register the Best Fit model with Azure ML Workspace

In [ ]:
best_run, fitted_model = local_run.get_output()
description = 'Quality prediction model Automl'
tags = None
model = local_run.register_model(description = description, tags = tags)

print(local_run.model_id) # This will be written to the script file later in the notebook.

# Download model from remote to cache

In [ ]:
model.get_model_path(model_name='AutoML7b4f55407best', version=1)

# Test the Model

In [ ]:
import pickle
import json
import numpy
from sklearn.externals import joblib
from azureml.core.model import Model

rawdata = '{ "data" : [1.999867,-18.0,4.999803,24.39346,28.1,35.9,39.5,23.1,986.1083,2084.438,2156.724,1097.541,20.70007,25.01674,26.33074,24.18593,21.69105,100.0,140.5465,132.7331,0.003690759,38.77173,24.99968,31.5868,0.1375652,26.95378,0.0,54.95482,0.01181043,60.0,40.0,13.0,0.03044727,0.7,0.1277504,0.460621,28283.33,26908.74,17991.27,18333.06640625] }'
testdata = json.loads(rawdata)['data']
testdata = [numpy.array(testdata)]

model_path = Model.get_model_path(model_name = local_run.model_id)
currentmodel = joblib.load(model_path)

result = currentmodel.predict(testdata)

print(testdata)
print(result)